# Face Recognition for the Indian Celebrity Using Facenet


FaceNet learns a neural network that encodes a face image into a vector of 128 numbers. By comparing two such vectors, you can then determine if two pictures are of the same person.
    

We will be using a pre-trained mode. The project is inspired from coursera course assignment.

Let's load the required packages. 


In [1]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.models import load_model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import dlib
import glob
from scipy.spatial import distance
from imutils import face_utils
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


## Load the pretrained model

In [2]:
model = faceRecoModel(input_shape=(3, 96, 96))

## Compile the model

Since we are just predicting the model's output so there is no need of loss function and accuracy which is required while evaluating the model. More about this can be found [here](https://stackoverflow.com/questions/46127625/need-to-compile-keras-model-before-model-evaluate)

In [3]:
load_weights_from_FaceNet(model)

In [4]:
print("Total Params:", model.count_params())
model.summary()

Total Params: 3743280
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 96, 96)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 3, 102, 102)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 48, 48)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 64, 48, 48)   256         conv1[0][0]                      
_______________________________________________________________________________________

In [5]:
model1 = load_model("facenet_keras_weight.h5")


/home/mukul/anaconda3/lib/python3.6/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
print(model1.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
__________________________________________________________________________________________________
Conv2d_2a_

### 3.2 - Face Recognition

1. Compute the target encoding of the image from image_path
2. Find the encoding from the database that has smallest distance with the target encoding. 
    - Initialize the `min_dist` variable to a large enough number (100). It will help you keep track of what is the closest encoding to the input's encoding.
    - Loop over the database dictionary's names and encodings.
        - Compute L2 distance between the target "encoding" and the current "encoding" from the database.
        - If this distance is less than the min_dist, then set min_dist to dist, and identity to name.

In [17]:
def recognize_face(face_descriptor, database):
    encoding = img_to_encoding(face_descriptor, model)
    min_dist = 100
    identity = None

    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():

        # Compute L2 distance between the target "encoding" and the current "emb" from the database.
        dist = np.linalg.norm(db_enc - encoding)

        print('distance for %s is %s' % (name, dist))

            #   If this distance is less than the min_dist, then set min_dist to dist, and identity to name
            #   less than equals to 80 Jahnavi Kapoor,less than equals to 154 Hritick,
            #   less than equals to 235 Alia Bhatt, less than equals to 317 Salman Khan,
            #   less than equals to 425 Shahrukh Khan, less than equals to 439 Aamir Khan,
        if dist < min_dist:
            min_dist = dist
            identity = name
    print(identity)
    print(min_dist)
    if int(identity) <= 80 and min_dist <=0.63:
        return ("Jahnavi Kapoor"), min_dist 
    if int(identity) > 80 and int(identity) <= 154:
        return ("Hritick Roushan"), min_dist
    if int(identity) > 154 and int(identity) <= 235:
        print(("Alia Bhatt"), min_dist)
        return ("Alia Bhatt"), min_dist
    if int(identity) > 235 and int(identity) <= 317:
        return ("Salman Khan"), min_dist
    if int(identity) > 317 and int(identity) <= 425:
        return ("Shahrukh Khan"), min_dist
    if int(identity) > 425 and int(identity) <= 439 :
        return ("Aamir Khan"), min_dist
       

In [18]:
detector = dlib.get_frontal_face_detector()

In [19]:
def img_path_to_encoding(image_path, model):
    img1 = cv2.imread(image_path, 1)
    return img_to_encoding(img1, model)
    

def img_to_encoding(image, model):
    try:
        image.shape
        print("checked for shape".format(image.shape))
        #image = cv2.resize(image, (160,160))#use this when model is model1
        image = cv2.resize(image, (96,96))#use this when model is model
        img = image[...,::-1]
        #img = np.around(img/255.0, decimals=12)#use this when model is model1
        img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)#use this when model is model
        x_train = np.array([img])
        #embedding = model1.predict(x_train)
        embedding = model.predict(x_train)
        return embedding
    except AttributeError:
        print("shape not found")

In [20]:
def initialize():
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob("images1/*"):#to test we are justusing 4 images
        identity = os.path.splitext(os.path.basename(file))[0]
        #database[identity] = img_path_to_encoding(file, model1)
        database[identity] = img_path_to_encoding(file, model)
    return database

In [21]:
def recognize_from_video():
    database = initialize()
    cap = cv2.VideoCapture(0)
    while True:
        ret, img = cap.read()
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if(img.size == 0 or img_rgb.size == 0):
            continue
                
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        subjects = detector(gray, 0)
        for subject in subjects:
            try:
                extract_face_info(img, img_rgb, database)
            except AttributeError:
                print("shape not found")
                
        cv2.imshow('Recognizing faces', img)
        if cv2.waitKey(100) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [22]:
def recognize_from_image(image):
    database = initialize()
    img = cv2.imread(image,1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    extract_face_info(img, img_rgb, database)
    #cv2.imshow('Recognizing faces', img)

In [23]:
def extract_face_info(img, img_rgb, database):
    #img = cv2.imread(image,1)
    faces = detector(img_rgb)
    x, y, w, h = 0, 0, 0, 0
    if len(faces) > 0:
        for face in faces:
            (x, y, w, h) = face_utils.rect_to_bb(face)
            #print(img_rgb.shape)
            #print(img.shape)
            #print(x,y,w,h)
            cv2.rectangle(img, (x, y), (y + h, x + w), (0, 255, 0))
            image1 = img[y:y + h, x:x + w]
            try:
                image1.shape
                print(image1.shape)
                name, min_dist = recognize_face(image1, database)
            except AttributeError:
                print("shape not found")
            if min_dist < 0.59:
                cv2.putText(img, "Face : " + name, (x, y - 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
                cv2.putText(img, "Dist : " + str(min_dist), (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, ( 255,0, 0), 2)
            else:
                cv2.putText(img,  name, (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)

In [24]:
image = "/home/mukul/Desktop/machine_learning/deepLearning/celebrity_face_recognition/test_images/1.jpg"
img = cv2.imread(image,1)
recognize_from_image(image)

checked for shape
checked for shape
checked for shape
checked for shape
checked for shape
checked for shape
(180, 179, 3)
checked for shape
distance for 427 is 0.8167752
distance for 14 is 0.96953374
distance for 237 is 0.97853446
distance for 156 is 0.7926242
distance for 326 is 1.2583225
distance for 82 is 1.1661737
156
0.7926242
Alia Bhatt 0.7926242


In [ ]:
recognize_from_video()